Привет, у меня трудности по преобразованию данных, неполучается подготовить их чтобы начать тренировать модель. Мне кажется проблема в типе данных, но я не уверен что именно. Если можно немого фидбека, Спасибо!!!

# Project with BERT
reference (https://gist.github.com/albahnsen/b02d2183c93067e3f248a428430c970e)
# Project Goal (Цель Проекта) 
We are given a file with Twitter comments. Those comments are marked as positive/negative. We need to build a NLP model that will be able to predict the "toxicity" of comments with a minimum of 0.75 F1 Score. <br/><br/>
Нам дан файл с комментариями с Твитера. Они помеченны как позитивные/негативные. Нам требуется построить NLP модель которая сможет предсказать токсичность комментариев с минимальной оценкой 0.75 по F1 Score.

# Project Outline (План Проекта)
1. Project Goal (Цель Проекта)
2. Outline  (План)
3. Libraries/Data Load  (Библеотеки/Выгрузка Данных)
4. Data Inspection/Data Preprocessing  (Исследовательский Анализ/Предобработка данных)
5. Training Pipeline + Logistic Regression  (Фунция для Оптимизяции Обучения + Простой Регрессор)
6. Try Out the Training Pipeline with Other Logistic ML Models  (Исследование Других Моделей)
7. Results  (Вывод)


# Libraries/Data Load  (Библеотеки/Выгрузка Данных)

In [1]:
import numpy as np
import pandas as pd

import os
from time import time

import torch
from transformers import BertTokenizer, BertModel
from transformers import logging

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [2]:
logging.set_verbosity_warning()
logging.set_verbosity_error()

In [3]:
local_path = './datasets/toxic_comments.csv'
web_path = '/datasets/toxic_comments.csv'

In [4]:
if os.path.exists(local_path):
  df = pd.read_csv(local_path)
elif os.path.exists(web_path):
  df = pd.read_csv(web_path)

# Data Inspection/Data Preprocessing 
# (Исследовательский Анализ/Предобработка данных)

Visual inspection of data showed that most of the comments are of english language. So far we can assume we will be using BERT tokenizer and BERT model "bert-base-uncased". <br/><br/>
После визуального исследования данных, ясно что мы работаем с английскими комментариями. Значит будем исползовать 'bert-base-uncased'. 

In [5]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [6]:
df = df.drop(columns=['Unnamed: 0'])

In [7]:
df.shape

(159292, 2)

It is also important to inspect the ratio of "positive/negative" target feature, we need to have a good balance while training the model 50/50. <br/><br/>
Надо тоже имметь ввиду распределения "позитивные/негативные" варианты для целегого признака. Нам нужно хорошее распределение 50/50.

In [8]:
print("toxic to all comments ratio: {}".format(df.toxic.sum()/df.shape[0]))

toxic to all comments ratio: 0.10161213369158527


The ratio issue will be solved in the training pipeline. <br/><br/>
Проблемма соотношения будет решенна в функции обучения.

# Training Pipeline + Logistic Regression  
# (Фунция для Оптимизяции Обучения + Простой Регрессор)

Optimizing training process - iterative model training and testing will let us know when to stop vectorizing the data.
In order to do that we need to create an instance of class that can hold vectorized train/test data, and when needed to perform those vectorizations in batch format. <br/><br/>
Оптимизация обучения - постепенное обучение и тестирование модели поможет нам понять когда можно перестать векторизировать данные. Для этого мы создадим класс который будет держать векторизированные данные, и векторизировать их по надобности, в зависсимости от размера батча.

--

We touched on the problem of positive/negative target feature ratio. Our class will handle it by first processing positive/negative data with 50/50 ratio, then if needed processing the rest of the data. <br/><br/>
Проблема с соотношением позитивного/негативного соотношения целегого признака будет решенна следующим образом: сначала обрабатываем все данные с соотношением 50/50 потом обрабатывем и подаем все осталные данные.

In [48]:
def extracting_embeddings(text):
      tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
      model = BertModel.from_pretrained('bert-base-uncased')
      model.eval()

      marked_text = "[CLS] " + text + " [SEP]"

      # Tokenize our sentence with the BERT tokenizer
      tokenized_text = tokenizer.tokenize(marked_text)
      
      # Truncate the text
      truncated_text = tokenized_text[:512]

      # Map the token strings to their vocabulary indices
      indexed_tokens = tokenizer.convert_tokens_to_ids(truncated_text)

      # Pad the text
      indexed_tokens = indexed_tokens + [0] * (512 - len(indexed_tokens))

      # Attention mask
      attention_mask = np.where(np.array(indexed_tokens) != 0, 1, 0)

      # Convert inputs to PyTorch tensors
      tokens_tensor = torch.tensor([np.array(indexed_tokens)])
      attention_mask = torch.tensor([np.array(attention_mask)])

      # Predict hidden states features for each layer
      with torch.no_grad():
        encoded_layers = model(tokens_tensor, attention_mask=attention_mask)

      return np.concatenate(encoded_layers.last_hidden_state.numpy())

In [57]:
hello = extracting_embeddings('hello')

In [61]:
hello

array([[-0.3060987 ,  0.2622295 , -0.18961759, ..., -0.1651108 ,
         0.10138296,  0.41190478],
       [-0.73901165, -0.03363602,  0.39322147, ..., -0.18180817,
        -0.18388468, -0.21847823],
       [ 0.58008885,  0.06270907, -0.26372138, ...,  0.39631116,
        -0.56836987, -0.49243015],
       ...,
       [-0.6366559 , -0.04542178,  0.40899047, ..., -0.01840694,
         0.20758131,  0.04689681],
       [-0.7131955 , -0.13211566,  0.29559928, ..., -0.00647255,
         0.280729  , -0.02023812],
       [-0.47300822, -0.00674867,  0.48973006, ...,  0.02761357,
         0.19655119,  0.03328196]], dtype=float32)

In [52]:
class BertDataPipeline:
  def __init__(self, df, batch_size):
    self.batch_size = batch_size
    self.current_batch = 0

    # optimize data for the ratio
    positive = df[df.toxic == 1]
    negative = df[df.toxic == 0]
    negative_in = negative.loc[:positive.shape[0]]
    negative_out = negative.loc[positive.shape[0]:]
    balanced = pd.concat([positive, negative_in], axis=0).reset_index(drop=True)
    balanced = balanced.sample(frac=1).reset_index(drop=True)
    self.df = pd.concat([balanced, negative_out], axis=0).reset_index(drop=True) # the balanced will come first

    self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    self.model = BertModel.from_pretrained('bert-base-uncased')
    self.model.eval()

    self.train = pd.DataFrame({'vectorized':[], 'toxic':[]})
    self.test = pd.DataFrame({'vectorized':[], 'toxic':[]})
  
  def data_train_new(self):
    def extracting_embeddings(text):
      marked_text = "[CLS] " + text + " [SEP]"

      # Tokenize our sentence with the BERT tokenizer
      tokenized_text = self.tokenizer.tokenize(marked_text)
      
      # Truncate the text
      truncated_text = tokenized_text[:512]

      # Map the token strings to their vocabulary indices
      indexed_tokens = self.tokenizer.convert_tokens_to_ids(truncated_text)

      # Pad the text
      indexed_tokens = indexed_tokens + [0] * (512 - len(indexed_tokens))

      # Attention mask
      attention_mask = np.where(np.array(indexed_tokens) != 0, 1, 0)

      # Convert inputs to PyTorch tensors
      tokens_tensor = torch.tensor([np.array(indexed_tokens)])
      attention_mask = torch.tensor([np.array(attention_mask)])

      # Predict hidden states features for each layer
      with torch.no_grad():
        encoded_layers = self.model(tokens_tensor, attention_mask=attention_mask)

      return np.concatenate(encoded_layers.last_hidden_state.numpy())

    # train data
    df_slice_train = self.df.loc[self.batch_size*self.current_batch:self.batch_size*(self.current_batch + 1)]
    self.current_batch += 1

    x_train = df_slice_train.text
    x_train = x_train.apply(lambda x: extracting_embeddings(x))
    y_train = df_slice_train.toxic

    self.train = pd.concat([self.train, pd.DataFrame({'vectorized': x_train, 'toxic': y_train})], axis=0).reset_index(drop=True)
    
    # test data
    df_slice_test = self.df.loc[self.batch_size*self.current_batch:self.batch_size*(self.current_batch + 1)]
    self.current_batch += 1

    x_test = df_slice_test.text
    x_test = x_test.apply(lambda x: extracting_embeddings(x))
    y_test = df_slice_test.toxic

    self.test = pd.concat([self.test, pd.DataFrame({'vectorized': x_test, 'toxic': y_test})], axis=0).reset_index(drop=True)

    # return new training data
    return pd.DataFrame({'vectorized':x_train, 'toxic':y_train})

  def data_train(self):
    return self.train
  
  def data_test(self):
    return self.test    

In [53]:
def train_model(bertDataPipeline, model, limit_seconds, f1_score_min):
  time_start = time()
  f1_score_best = 0
  initial_learn = True

  while (time() - time_start < limit_seconds):
    # exit condition - meet the score requirements, or time limit ^^^
    # (also if we run out of data - unlikely, not to worry about, it would also take too long to vectorize all that data)

    if f1_score_best >= f1_score_min:
      break

    if initial_learn:
      initial_learn = False
      if bertDataPipeline.data_train().shape[0] > 0:
        df_train = bertDataPipeline.data_train()
        print(df_train.head())
        model.fit(df_train.vectorized, df_train.toxic)
        df_test = bertDataPipeline.data_test()
        z_test = model.predict(df_test.vectorized)
        y_test = df_test.toxic
        f1_score_current = f1_score(y_test, z_test)
        if f1_score_current > f1_score_best:
          f1_score_best = f1_score_current
    else:
      df_train = bertDataPipeline.data_train_new()
      print(df_train.head())
      model.fit(df_train.vectorized, df_train.toxic)
      df_test = bertDataPipeline.data_test()
      z_test = model.predict(df_test.vectorized)
      y_test = df_test.toxic
      f1_score_current = f1_score(y_test, z_test)
      if f1_score_current > f1_score_best:
        f1_score_best = f1_score_current
  
  print('best f1_score {}'.format(f1_score_best))
  print('time to train {}s'.format(time() - time_start))

In [54]:
bertDataPipeline = BertDataPipeline(df, 5)

In [55]:
logistic_linear_model = LogisticRegression(max_iter=1000)

In [56]:
train_model(bertDataPipeline, logistic_linear_model, 120, 0.75)

                                          vectorized  toxic
0  [[-0.107602015, 0.42301303, 0.07229452, -0.567...      0
1  [[0.19171593, -0.09149636, -0.33626673, -0.082...      1
2  [[0.117458366, -0.10120446, -0.13708822, -0.56...      1
3  [[-0.9321038, 0.0019665472, -0.0461475, -0.515...      0
4  [[-0.15104553, 0.081879444, -0.080407724, -0.5...      1


ValueError: setting an array element with a sequence.